In [94]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np

In [95]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [96]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [97]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using mps device


In [98]:
# add a custom layer
class CustomDense(nn.Module):

    def __init__(self, size_in, size_out):
        super().__init__()

        self.weights = nn.Parameter(
            torch.Tensor(size_out, size_in)
        )  
        # nn.Parameter - trainable parameters, it's what we send to the constructor of the Optimizier.
        # serach for model.parameters() 

        self.bias = nn.Parameter(
            torch.Tensor(size_out)
        ) 

        # initialize weights and biases

        # He initialization
        nn.init.kaiming_uniform_(self.weights, a=np.sqrt(5))
        nn.init.uniform_(self.bias, -0.1, 0.1)

    def forward(self, x):
        w_times_x= torch.mm(x, self.weights.t())
        return torch.add(w_times_x, self.bias)  # w times x + b

In [99]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            CustomDense(28*28, 512), # instead of nn.Linear
            nn.ReLU(),
            CustomDense(512, 512),
            nn.ReLU(),
            CustomDense(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# sending the model to device must be done before initializing the optimizer
# otherwise it won't update the parameters
model = NeuralNetwork().to(device)
print(model)

#model = torch.compile(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): CustomDense()
    (1): ReLU()
    (2): CustomDense()
    (3): ReLU()
    (4): CustomDense()
  )
)


In [100]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [101]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    # make it faster, no gradient computation
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [102]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")



Epoch 1
-------------------------------
loss: 2.312792  [   64/60000]
loss: 2.292289  [ 6464/60000]
loss: 2.275900  [12864/60000]
loss: 2.270404  [19264/60000]
loss: 2.252733  [25664/60000]
loss: 2.232029  [32064/60000]
loss: 2.232559  [38464/60000]
loss: 2.206539  [44864/60000]
loss: 2.198829  [51264/60000]
loss: 2.174195  [57664/60000]
Test Error: 
 Accuracy: 49.1%, Avg loss: 2.163399 

Epoch 2
-------------------------------
loss: 2.173644  [   64/60000]
loss: 2.162490  [ 6464/60000]
loss: 2.105308  [12864/60000]
loss: 2.123774  [19264/60000]
loss: 2.072922  [25664/60000]
loss: 2.013977  [32064/60000]
loss: 2.036089  [38464/60000]
loss: 1.964644  [44864/60000]
loss: 1.960953  [51264/60000]
loss: 1.898842  [57664/60000]
Test Error: 
 Accuracy: 56.9%, Avg loss: 1.892512 

Epoch 3
-------------------------------
loss: 1.920631  [   64/60000]
loss: 1.895757  [ 6464/60000]
loss: 1.775648  [12864/60000]
loss: 1.820230  [19264/60000]
loss: 1.710505  [25664/60000]
loss: 1.655293  [32064/600

In [103]:

# save
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

# load
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))


Saved PyTorch Model State to model.pth


<All keys matched successfully>

In [104]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]



model.eval()
x, y = test_data[0][0], test_data[0][1]

# also with no autograd
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    print("Probabilities: ", nn.Softmax(dim=1)(pred))

    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')



Probabilities:  tensor([[0.0022, 0.0011, 0.0078, 0.0029, 0.0068, 0.2102, 0.0071, 0.2501, 0.1131,
         0.3987]], device='mps:0')
Predicted: "Ankle boot", Actual: "Ankle boot"


In [105]:
print("Total parameters: ", np.sum([np.product(x.shape)  for x in model.parameters()]))

Total parameters:  669706
